In [1]:
import requests
import pandas as pd
from random import randint
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import getpass

In [2]:
c_id = getpass.getpass()

········


In [3]:
c_secret = getpass.getpass()

········


In [4]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=c_id,client_secret=c_secret))

In [5]:
popvortex_100 = pd.read_csv('C:/Users/ebour/Documents/()_Ironhack DA Bootcamp/()_Labs/6_1_lab-web-scraping-single-page/popvortex_100.csv')

In [18]:
suggestion_database = pd.read_csv('clusterized_data.csv')

In [24]:
suggestion_database

,track_name,artist_name,cluster
0,The Joker And The Queen (feat. Taylor Swift),Ed Sheeran,2
1,The Joker And The Queen (feat. Taylor Swift),Taylor Swift,2
2,Zo Zijn,sor,0
3,Options (feat. 24kGoldn),Internet Money,0
4,Options (feat. 24kGoldn),24kGoldn,0
...,...,...,...
8616,Champagne Life,Ne-Yo,9
8617,O Let's Do It,Waka Flocka Flame,0
8618,Fistful of Tears,Maxwell,9
8619,Lemonade,Gucci Mane,3


# MVP

In [19]:
import pickle
filename = 'spotify_cluster_model.sav'
cluster_model = pickle.load(open(filename, 'rb'))

In [20]:
filename_scaler = 'spotify_scaler.sav'
scaler = pickle.load(open(filename_scaler, 'rb'))

In [34]:
def song_advice():
    song_input = input('Please fill in the name of a song you like :')
    song_input = song_input.lower()
    top_100_check = popvortex_100[popvortex_100['title'] == song_input]
    
    if len(top_100_check['title']) > 0 :
        i = randint(1,100)
        while popvortex_100['title'][i] == song_input:   # used to avoid advising the same song as the input
            i = randint(1,100)
        proposed_title = popvortex_100['title'][i]
        proposed_artist = popvortex_100['artist'][index]
        output = 'Great choice ! This song is in the PopVortex Top 100. You should listen to ' + proposed_title + ' by ' + proposed_artist + '. You might like it !'
    
    else :
        res = sp.search(q='track:' + song_input, type='track')
        data = pd.DataFrame(res['tracks']['items'])
        if len(data) == 0:
            output = 'We are sorry, but we cannot find any song with this name. Please try again.'
        else:
            data['name'] = data['name'].apply(lambda x: x.lower())
            data = data[data['name'] == song_input].reset_index(drop = True)
            track_id = None
            if len(data['artists'])>1:
                artist_input = input('Please fill in the name of the artist :')
                artist_input = artist_input.lower()
                for i in range(len(data['artists'])):
                    art = data['artists'][i][0]['name'].lower()
                    if art == artist_input:
                        track_id = data['id'][i]
                        break
            else:
                track_id =  data['id'][0]
            track_uri = 'spotify:track:' + track_id
            audio_features = pd.DataFrame(sp.audio_features(track_uri)[0], index=[0])[['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']]
#             audio_features = audio_features[['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']]
            audio_feat_prep = scaler.transform(audio_features)
            song_input_cluster = cluster_model.predict(audio_feat_prep)[0]
            suggestion_list = suggestion_database[suggestion_database['cluster'] == song_input_cluster].reset_index(drop=True)
            j = randint(1,len(suggestion_list['track_name']))
            while suggestion_list['track_name'][j] == song_input:  #used to avoid advising the same song as the input (if ever in the suggestion list)
                j = randint(1,len(suggestion_list['track_name']))
            proposed_title = suggestion_list['track_name'][j]
            proposed_artist = suggestion_list['artist_name'][j]
            output = 'Your song is not in the PopVortex Top 100. Our advice for you is: ' + proposed_title + ' by ' + proposed_artist + '. You might like it !'
        
    return output

In [35]:
song_advice()

Please fill in the name of a song you like :Poor Leno
Please fill in the name of the artist :Röyksopp


'Your song is not in the PopVortex Top 100. Our advice for you is: 123 (Dolly Song) feat. Karma Child by Rompasso. You might like it !'